# Intro to LLMstudio - proxy

This tutorial serves as an introduction to LLMstudio, a platform designed to facilitate interactions with large language models (LLMs) through a proxy server. By following this guide, users will learn how to set up and run a proxy server that acts as an intermediary between the client and the LLM provider, ensuring seamless communication and data handling. The notebook demonstrates the process of initializing the server, configuring the proxy, and making requests to the LLM, providing a comprehensive overview of the system's capabilities.

Additionally, the tutorial highlights the integration of tracking features within LLMstudio, allowing users to monitor and log interactions with the LLMs. This is particularly useful for analyzing performance metrics, such as latency and token usage, which can help in optimizing the model's responses and understanding its behavior in different scenarios. By the end of this tutorial, users will have a solid foundation in setting up and utilizing LLMstudio for efficient and effective LLM interactions.



## start proxy server


In [1]:
from llmstudio.server import start_servers
start_servers()

Running LLMstudio Proxy on http://0.0.0.0:8001 
Running LLMstudio Tracking on http://0.0.0.0:8002 


In [2]:
from llmstudio_proxy.provider import LLMProxyProvider as LLM
from llmstudio_proxy.provider import ProxyConfig

llm = LLM(provider="openai", 
          proxy_config=ProxyConfig(host="0.0.0.0", port="8001"))

Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [3]:
result = llm.chat("olá", model="gpt-4o")


In [4]:
result.chat_output, result.metrics

('Olá! Como posso ajudar você hoje?',
 {'input_tokens': 2,
  'output_tokens': 11,
  'total_tokens': 13,
  'cost_usd': 0.000175,
  'latency_s': 1.154630184173584,
  'time_to_first_token_s': 1.0609641075134277,
  'inter_token_latency_s': 0.00980022218492296,
  'tokens_per_second': 8.66078172653819})

### Use the LLMStudio sdk entrypoint


In [12]:
from llmstudio.providers import LLM
from pprint import pprint

In [6]:
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
from llmstudio_proxy.provider import ProxyConfig
proxy = ProxyConfig(host="0.0.0.0", port="8001")
print(proxy)

openai = LLM("openai", proxy_config=proxy)


host='0.0.0.0' port='8001' url=None username=None password=None
Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [7]:
openai._provider


### Chat (non-stream)

In [8]:
openai.chat("What's your name", model="gpt-4o")

ChatCompletion(id='fde9001a-c9fb-4a52-b23e-a3002d53a3c0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I’m called ChatGPT. How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729178752, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output='I’m called ChatGPT. How can I assist you today?', chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729178753.123771, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 14, 'total_tokens': 18, 'cost_usd': 0.00023, 'latency_s': 0.8865439891815186, 'time_to_first_token_s': 0.7208070755004883, 'inter_token_latency_s': 0.011822138513837541, 'tokens_per_second': 16.919634200947442})

#### Async version

In [9]:
await openai.achat("What's your name", model="gpt-4o")

### Chat (stream)

In [10]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, the vast and enigmatic expanse that stretches beyond our planet, has captivated human imagination for

 centuries. It is a boundless arena, filled with billions of galaxies, each containing countless stars and

 planets, many still shrouded in mystery. The infinite darkness of space is punctuated by the

 brilliant glow of stars and the swirling colors of nebulae, painting a picture of both incredible beauty

 and monumental scale. As humanity ventures further into this final frontier, fueled by advances in technology and a

 relentless curiosity, we aim to uncover the secrets of distant worlds, understand the fundamental forces that govern the

 universe, and perhaps find signs of life beyond Earth. Space exploration challenges us to push beyond our limits

, fostering innovation and cooperation on a global scale, and inspiring generations to look up and dream of what

 lies beyond our sky.

## Metrics:
{'cost_usd': 0.00253,
 'input_tokens': 8,
 'inter_token_latency_s': 0.011

#### Async version

In [11]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, the vast expanse beyond Earth's atmosphere, captivates the imagination and science alike with its

 boundless mysteries and breathtaking beauty. It is a realm of solitude and silence, where distant stars burn

 like ancient beacons of light and galaxies spiral gracefully across the cosmos. The vastness of space is

 not just about distance; it's also a window into time, as the light from celestial objects takes millions

, sometimes billions, of years to reach us, allowing astronomers to glimpse the universe's history.

 From the swirling clouds of nebulae to the enigmatic pull of black holes, space is a dynamic

 playground of physical laws that govern the existence of everything we know. As we continue to explore this celestial

 frontier, each new discovery deepens our understanding of the universe and our place within it, fostering a

 sense of wonder that inspires generations to reach for the stars.

## Metrics:
{'cost_usd': 0.00010559999999999999,
 'input_tokens': 8,
 'i